In [1]:
import numpy as np

### 단어 수준의 원-핫 인코딩하기(간단 예) 

In [2]:
samples = ['The cat sat on the mat.', 'The dog ate my homework.']
token_index = {}
for sample in samples:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index) + 1
max_length = 10
results = np.zeros(shape=(len(samples),
                         max_length,
                         max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i,j,index] = 1.

In [3]:
token_index

{'The': 1,
 'ate': 8,
 'cat': 2,
 'dog': 7,
 'homework.': 10,
 'mat.': 6,
 'my': 9,
 'on': 4,
 'sat': 3,
 'the': 5}

### 케라스를 사용한 단어수준의 원-핫 인코딩

In [4]:
from tensorflow import keras
from keras.preprocessing.text import Tokenizer

In [5]:
samples = ['The cat sat on the mat.', 'The dog ate my homework.']
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(samples)

sequences = tokenizer.texts_to_sequences(samples)

one_hot_results = tokenizer.texts_to_matrix(samples)
word_index = tokenizer.word_index

print(f'{len(word_index)}개의 고유한 토큰을 찾았습니다.')
word_index

9개의 고유한 토큰을 찾았습니다.


{'ate': 7,
 'cat': 2,
 'dog': 6,
 'homework': 9,
 'mat': 5,
 'my': 8,
 'on': 4,
 'sat': 3,
 'the': 1}

### Embedding IMDB 문제에 적용해보기

#### Embedding 층을 사용하여 단어 임베딩 학습하기

In [6]:
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences

In [7]:
max_feature = 1000
maxlen = 20

(X_train, y_train),(X_test, y_test) = imdb.load_data(
    num_words = max_feature
)

X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding

In [9]:
model = Sequential([
                    Embedding(10000,8,input_length=maxlen),
                    Flatten(),
                    Dense(1,activation='sigmoid')
])
model.compile(loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 8)             80000     
_________________________________________________________________
flatten (Flatten)            (None, 160)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


In [10]:
history = model.fit(X_train,y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 2s 2ms/step - loss: 0.6877 - acc: 0.5431 - val_loss: 0.6322 - val_acc: 0.6836
Epoch 2/10
625/625 [==============================] - 1s 2ms/step - loss: 0.5928 - acc: 0.7169 - val_loss: 0.5480 - val_acc: 0.7174
Epoch 3/10
625/625 [==============================] - 1s 2ms/step - loss: 0.5128 - acc: 0.7510 - val_loss: 0.5292 - val_acc: 0.7264
Epoch 4/10
625/625 [==============================] - 1s 2ms/step - loss: 0.4832 - acc: 0.7692 - val_loss: 0.5256 - val_acc: 0.7328
Epoch 5/10
625/625 [==============================] - 1s 2ms/step - loss: 0.4717 - acc: 0.7741 - val_loss: 0.5268 - val_acc: 0.7340
Epoch 6/10
625/625 [==============================] - 1s 2ms/step - loss: 0.4619 - acc: 0.7777 - val_loss: 0.5277 - val_acc: 0.7350
Epoch 7/10
625/625 [==============================] - 1s 2ms/step - loss: 0.4562 - acc: 0.7814 - val_loss: 0.5303 - val_acc: 0.7350
Epoch 8/10
625/625 [==============================] - 1s 2ms/step - loss: 0.

#### 사전 훈련된 단어 임베딩 사용하기

In [ ]:
# from google.colab import files
# fileuploaded = files.upload()

Saving aclImdb.zip to aclImdb.zip


In [ ]:
# filename = list(fileuploaded.keys())[0]

In [16]:
!cp ./drive/MyDrive/Colab\ Notebooks/data/Deep-learning/aclImdb.zip . 

In [17]:
!mkdir aclImdb

In [19]:
!unzip -qq aclImdb.zip -d aclImdb

In [35]:
import os
imdb_dir = './aclImdb/aclImdb'
train_dir = os.path.join(imdb_dir,'train')

labels = []
texts = []

for label_type in ['neg','pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            with open(os.path.join(dir_name,fname), encoding='utf8') as f:
                texts.append(f.read())    
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [38]:
len(texts), len(labels)

(25000, 25000)

In [39]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('%s개의 고유한 토큰을 찾았습니다.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('데이터 텐서의 크기:', data.shape)
print('레이블 텐서의 크기:', labels.shape)

# 데이터를 훈련 세트와 검증 세트로 분할합니다.
# 샘플이 순서대로 있기 때문에 (부정 샘플이 모두 나온 후에 긍정 샘플이 옵니다) 
# 먼저 데이터를 섞습니다.
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

88582개의 고유한 토큰을 찾았습니다.
데이터 텐서의 크기: (25000, 100)
레이블 텐서의 크기: (25000,)


In [44]:
embedding_dim = 100

# embedding_matrix = np.zeros((max_words, embedding_dim))
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if i < max_words:
#         if embedding_vector is not None:
#             # 임베딩 인덱스에 없는 단어는 모두 0이 됩니다.
#             embedding_matrix[i] = embedding_vector

In [45]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, 8, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=100,
                    batch_size=32,
                    validation_data=(x_val, y_val))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 8)            80000     
_________________________________________________________________
flatten_3 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                25632     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 105,665
Trainable params: 105,665
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
7/7 [==============================] - 1s 79ms/step - loss: 0.6942 - acc: 0.4478 - val_loss: 0.6931 - val_acc: 0.5058
Epoch 2/100
7/7 [==============================] - 0s 58ms/step - loss: 0.6544 - acc: 0.9461 - val_loss: 0.6929 - va